<a href="https://colab.research.google.com/github/adolfoguimaraes/inteligenciaartificial/blob/main/code/11_BayesNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pgmpy

In [ ]:
# Imports
import pandas as pd
from pgmpy.estimators import PC
from pgmpy.inference import VariableElimination
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.estimators import BayesianEstimator


# Redes Bayesianas utilizando Python 

Na aula de hoje, vamos trabalhar com exemplos de Redes Bayesianas utilizando a linguagem Python. Para isso, vamos usar uma biblioteca que possui algoritmos com esse propósito: a PGMPY. Para instalar, execute o comando de instalação `pip install pgmpy`.

## O exemplo de Monty Hall

Vamos ver como utilizamos esse problema para estimar as probabilidades no problema de Monty Hall. A rede a seguir mostra esse exemplo: 

<img src="https://pgmpy.org/_images/examples_Monty_Hall_Problem_3_0.png" width="20%" />

onde P é o prêmio, C é o competidor e H é o apresentador. O prêmio foi colocado aleatoriamente em uma das portas, logo: $P(P=1) = P(P=2) = P(P=3) = \frac{1}{3}$. Da mesma forma, o competidor escolhe a porta de forma aleatória, então: $P(P=1) = P(P=2) = P(P=3) = \frac{1}{3}$. Já a escolha do apresentado depende de onde o prêmio está e de qual porta o competidor vai escolher. Com isso, temos a seguinte distribuição de probabilidade: 

```
P(C):
+----------+----------+-----------+-----------+
|    C     |     0    |     1     |      2    |
+----------+----------+-----------+-----------+
|          |    0.33  |    0.33   |    0.33   |
+----------+----------+-----------+-----------+

P(P):
+----------+----------+-----------+-----------+
|    P     |     0    |     1     |      2    |
+----------+----------+-----------+-----------+
|          |    0.33  |    0.33   |    0.33   |
+----------+----------+-----------+-----------+

P(H | P, C):
+------+------+------+------+------+------+------+------+------+------+
|   C  |          0         |          1         |          2         |
+------+------+------+------+------+------+------+------+------+------+
|   P  |   0  |   1  |   2  |   0  |   1  |   2  |   0  |   1  |   2  |
+------+------+------+------+------+------+------+------+------+------+
|  H=0 |   0  |   0  |   0  |   0  |  0.5 |   1  |   0  |   1  |  0.5 |
+------+------+------+------+------+------+------+------+------+------+
|  H=1 |  0.5 |   0  |   1  |   0  |   0  |   0  |   1  |   0  |  0.5 |
+------+------+------+------+------+------+------+------+------+------+
|  H=2 |  0.5 |   1  |   0  |   1  |  0.5 |   0  |   0  |   0  |   0  |
+------+------+------+------+------+------+------+------+------+------+
```

In [ ]:


# Defining the network structure
model = BayesianNetwork([("C", "H"), ("P", "H")])

# Defining the CPDs:
cpd_c = TabularCPD("C", 3, [[0.33], [0.33], [0.33]])
cpd_p = TabularCPD("P", 3, [[0.33], [0.33], [0.33]])
cpd_h = TabularCPD(
    "H",
    3,
    [
        [0, 0, 0, 0, 0.5, 1, 0, 1, 0.5],
        [0.5, 0, 1, 0, 0, 0, 1, 0, 0.5],
        [0.5, 1, 0, 1, 0.5, 0, 0, 0, 0],
    ],
    evidence=["C", "P"],
    evidence_card=[3, 3],
)

# Associating the CPDs with the network structure.
model.add_cpds(cpd_c, cpd_p, cpd_h)

model.check_model()

True

In [ ]:
# Some other methods
print(model.get_cpds('H'))

+------+------+------+------+------+------+------+------+------+------+
| C    | C(0) | C(0) | C(0) | C(1) | C(1) | C(1) | C(2) | C(2) | C(2) |
+------+------+------+------+------+------+------+------+------+------+
| P    | P(0) | P(1) | P(2) | P(0) | P(1) | P(2) | P(0) | P(1) | P(2) |
+------+------+------+------+------+------+------+------+------+------+
| H(0) | 0.0  | 0.0  | 0.0  | 0.0  | 0.5  | 1.0  | 0.0  | 1.0  | 0.5  |
+------+------+------+------+------+------+------+------+------+------+
| H(1) | 0.5  | 0.0  | 1.0  | 0.0  | 0.0  | 0.0  | 1.0  | 0.0  | 0.5  |
+------+------+------+------+------+------+------+------+------+------+
| H(2) | 0.5  | 1.0  | 0.0  | 1.0  | 0.5  | 0.0  | 0.0  | 0.0  | 0.0  |
+------+------+------+------+------+------+------+------+------+------+


In [ ]:
infer = VariableElimination(model)
posterior_p = infer.query(["P"], evidence={"C": 0, "H": 2})
print(posterior_p)

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

+------+----------+
| P    |   phi(P) |
+======+==========+
| P(0) |   0.3333 |
+------+----------+
| P(1) |   0.6667 |
+------+----------+
| P(2) |   0.0000 |
+------+----------+


## Mais um exemplo

Para este exemplo, vamos utilizar uma estrutura de grafo e distribuição de probabilidades conhecidas. O exemplo a seguir foi retirado da própria documentação da biblioteca. Vamos trabalhar com o exemplo a seguir: 

<img src="https://pgmpy.org/_images/detailed_notebooks_1._Introduction_to_Probabilistic_Graphical_Models_22_0.png" width="60%" />

Com a seguinte distribuição de probabilidade: 

<img src="https://pgmpy.org/_images/detailed_notebooks_2._Bayesian_Networks_4_0.png" width="70%" />

In [ ]:
model = BayesianNetwork([('D','G'),('I','G'),('G','L'),('I','S')])

cpd_d = TabularCPD(variable='D', variable_card=2, values=[[0.6],[0.4]], state_names={'D': ['Easy','Hard']})
cpd_i = TabularCPD(variable='I', variable_card=2, values=[[0.7],[0.3]], state_names={'I': ['Dumb','Intelligent']})

cpd_g = TabularCPD(variable='G', variable_card=3,
                   values=[[0.3, 0.05, 0.9,  0.5],
                           [0.4, 0.25, 0.08, 0.3],
                           [0.3, 0.7,  0.02, 0.2]],
                  evidence=['I', 'D'],
                  evidence_card=[2, 2],
                  state_names={'G': ['A','B','C'], 'I': ['Dumb','Intelligent'], 'D': ['Easy', 'Hard']})

cpd_l = TabularCPD(variable='L', variable_card=2,
                   values=[[0.1, 0.4, 0.99],
                           [0.9, 0.6, 0.01]],
                   evidence=['G'],
                   evidence_card=[3],
                   state_names={'L': ['Bad','Good'], 'G': ['A','B','C']})

cpd_s = TabularCPD(variable='S', variable_card=2,
                   values=[[0.95, 0.2],
                           [0.05, 0.8]],
                   evidence=['I'],
                   evidence_card=[2],
                   state_names={'S': ['Bad','Good'], 'I': ['Dumb', 'Intelligent']})

# Associating the CPDs with the network
model.add_cpds(cpd_d, cpd_i, cpd_g, cpd_l, cpd_s)

# check_model checks for the network structure and CPDs and verifies that the CPDs are correctly
# defined and sum to 1.
model.check_model()

True

In [ ]:
model.get_cpds()

[<TabularCPD representing P(D:2) at 0x7f842a566fa0>,
 <TabularCPD representing P(I:2) at 0x7f842a566d00>,
 <TabularCPD representing P(G:3 | I:2, D:2) at 0x7f842a566fd0>,
 <TabularCPD representing P(L:2 | G:3) at 0x7f842a566cd0>,
 <TabularCPD representing P(S:2 | I:2) at 0x7f842a566b80>]

In [ ]:
print(model.get_cpds('G'))

+------+---------+---------+----------------+----------------+
| I    | I(Dumb) | I(Dumb) | I(Intelligent) | I(Intelligent) |
+------+---------+---------+----------------+----------------+
| D    | D(Easy) | D(Hard) | D(Easy)        | D(Hard)        |
+------+---------+---------+----------------+----------------+
| G(A) | 0.3     | 0.05    | 0.9            | 0.5            |
+------+---------+---------+----------------+----------------+
| G(B) | 0.4     | 0.25    | 0.08           | 0.3            |
+------+---------+---------+----------------+----------------+
| G(C) | 0.3     | 0.7     | 0.02           | 0.2            |
+------+---------+---------+----------------+----------------+


In [ ]:
infer = VariableElimination(model)

In [ ]:
g_dist = infer.query(['G'])
print(g_dist)

Eliminating: D: 100%|██████████| 2/2 [00:00<00:00, 257.07it/s]

+------+----------+
| G    |   phi(G) |
+======+==========+
| G(A) |   0.3620 |
+------+----------+
| G(B) |   0.2884 |
+------+----------+
| G(C) |   0.3496 |
+------+----------+


In [ ]:
print(infer.query(['G'], evidence={'D': 'Easy', 'I': 'Intelligent'}))

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

+------+----------+
| G    |   phi(G) |
+======+==========+
| G(A) |   0.9000 |
+------+----------+
| G(B) |   0.0800 |
+------+----------+
| G(C) |   0.0200 |
+------+----------+


In [ ]:
print(infer.query(['G'], evidence={'S': 'Bad', 'L': 'Bad', 'I': 'Dumb'}))

Eliminating: D: 100%|██████████| 1/1 [00:00<00:00, 174.97it/s]

+------+----------+
| G    |   phi(G) |
+======+==========+
| G(A) |   0.0327 |
+------+----------+
| G(B) |   0.2224 |
+------+----------+
| G(C) |   0.7448 |
+------+----------+


## Usando uma base de dados como entrada. 

In [ ]:
# Ler os arquivos 
df_cdq = pd.read_csv("../datasets/bayes_network/CDQ_J.csv")
df_hsq = pd.read_csv("../datasets/bayes_network/HSQ_J.csv")
df_ocq = pd.read_csv("../datasets/bayes_network/OCQ_J.csv")

# Selecionar as variáveis que interessam
df_cdq = df_cdq[['SEQN','CDQ010']]
df_hsq = df_hsq[['SEQN','HSQ500','HSQ510']]
df_ocq = df_ocq[['SEQN','OCQ180']]

df_cdq.dropna(inplace=True)
df_hsq.dropna(inplace=True)
df_ocq.dropna(inplace=True)

df_outer = pd.merge(df_cdq, df_hsq, on="SEQN", how='outer')
df_outer = pd.merge(df_outer, df_ocq, on="SEQN", how='outer')

df_outer.dropna(inplace=True)


In [ ]:
finalData = df_outer.rename(columns={'CDQ010': 'A', 'HSQ500': 'B', 'OCQ180': 'C', 'HSQ510': 'D'})
finalData.drop(['SEQN'],axis=1, inplace=True)
finalData.head()

,A,B,D,C
0,2.0,2.0,2.0,30.0
3,2.0,2.0,2.0,35.0
7,2.0,2.0,2.0,40.0
9,2.0,2.0,2.0,5.0
10,2.0,2.0,2.0,40.0


In [ ]:
finalData.loc[finalData['C'] < 30, 'C'] = 0
finalData.loc[((finalData['C'] < 60) & (finalData['C'] >= 30)), 'C'] = 1
finalData.loc[finalData['C'] >= 60, 'C'] = 2

In [ ]:
finalData.head()

,A,B,D,C
0,2.0,2.0,2.0,1.0
3,2.0,2.0,2.0,1.0
7,2.0,2.0,2.0,1.0
9,2.0,2.0,2.0,0.0
10,2.0,2.0,2.0,1.0


In [ ]:
est = PC(data=finalData)
model_est = est.estimate(variant='stable', max_cond_vars=2)

Working for n conditional variables: 2: 100%|██████████| 2/2 [00:00<00:00, 17.79it/s]


In [ ]:
print(model_est.edges)

[('A', 'D'), ('B', 'D')]


In [ ]:
model_ = BayesianNetwork(model_est.edges())
model_.fit(finalData)

In [ ]:
model_.get_cpds()

[<TabularCPD representing P(A:3) at 0x7f842afc1580>,
 <TabularCPD representing P(D:3 | A:3, B:3) at 0x7f842a497850>,
 <TabularCPD representing P(B:3) at 0x7f842a490d00>]

In [ ]:
infer = VariableElimination(model_)

In [ ]:
d_dist = infer.query(['D'])
print(d_dist)

Eliminating: A: 100%|██████████| 2/2 [00:00<00:00, 204.50it/s]

+--------+----------+
| D      |   phi(D) |
+========+==========+
| D(1.0) |   0.0556 |
+--------+----------+
| D(2.0) |   0.9435 |
+--------+----------+
| D(9.0) |   0.0010 |
+--------+----------+


In [ ]:
print(infer.query(['D'], evidence={'A': 1.0}))

Eliminating: B: 100%|██████████| 1/1 [00:00<00:00, 319.18it/s]

+--------+----------+
| D      |   phi(D) |
+========+==========+
| D(1.0) |   0.0952 |
+--------+----------+
| D(2.0) |   0.9016 |
+--------+----------+
| D(9.0) |   0.0032 |
+--------+----------+


In [ ]:
print(infer.query(['D'], evidence={'A': 1.0, 'B': 1.0}))

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

+--------+----------+
| D      |   phi(D) |
+========+==========+
| D(1.0) |   0.1609 |
+--------+----------+
| D(2.0) |   0.8391 |
+--------+----------+
| D(9.0) |   0.0000 |
+--------+----------+


## Exercício

01 - Considere uma usina nuclear que possui um alarme que soa quando detecta que um medidor de temperatura excedeu um determinado limite. O medidor mede a temperatura do núcleo. Considere as variáveis booleanas **A** (som do alarme), **FA** (alarme está com defeito), **FG** (o medidor está com defeito) e os nós discretos multivariados **G** (leitura do medidor) e T (Temperatura do núcleo). Desenhe um rede baysiana para este domínio sabendo que o medidor tem maior probabilidade de falhar quando a temperatura central fica muito alta.

*Coloque sua resposta neste espaço* 

02 - Considere a seguinte rede bayesianas, a distribuição de probabiliadades e responda o que se pede: 

<img src="https://static.javatpoint.com/tutorial/ai/images/bayesian2.png">

**Tradução**

* *Burglary*: roubo
* *Earthquake*: terremoto
* *Alarm*: alarme
* *David calls*: David ligar 
* *Sophia calls*: Sophia ligar

**Questões**

a)Qual a probabilidade do alarme tocar, dado que está acontecendo um roubo? 

b) Qual a probabilidade de David ligar, dado que o alarme tocou e está tendo um terremoto?

c) Qual a probabililade de Sophia ligar, dado que o alarme tocou e está tendo um roubo? 

In [ ]:
# Insira código e respostas a  partir deste espaço